In [1]:
import vedo
import inspect
from functools import wraps
import numpy as np
import os
import pandas as pd
import yaml

In [3]:
def napari_compatible(func):
    sig = inspect.signature(func)
    params = list(sig.parameters.values())
    
    # Change the type annotation of all arguments (not keyword arguments) to SurfaceData
    for i, param in enumerate(params):
        if param.default == param.empty:
            params[i] = inspect.Parameter(f'Input_mesh_{i}', param.kind, annotation='napari.types.SurfaceData')
            
    new_sig = sig.replace(parameters=params)
    
    @wraps(func)
    def wrapper(surface_data, *args, **kwargs):

        mesh = vedo.Mesh(surface_data)
        result = func(mesh, *args, **kwargs)
        return (result.points(), np.asarray(result.faces()))
    
    wrapper.__signature__ = new_sig
    return wrapper
        

In [5]:
functions_dict = [x for x  in inspect.getmembers(vedo.Mesh) if x[0] == '__dict__'][0][1]
function = functions_dict['decimate']
compatible_function = napari_compatible(function)
compatible_function

<function vedo.mesh.Mesh.decimate(Input_mesh_0: 'napari.types.SurfaceData', fraction=0.5, n=None, preserve_volume=True, regularization=0.0) -> 'Mesh'>

In [6]:
signature = inspect.signature(compatible_function)
signature.return_annotation

'Mesh'

In [7]:
class VedoFunctions:
    pass

# Create an instance of the Functions class
functions = VedoFunctions()

# keep track of the functions that are added
functions_to_add_to_yaml = []

In [8]:
for name, func in functions_dict.items():
    if not callable(func):
        print(f'{name} is not a callable')
        continue

    signature = inspect.signature(func)
    if signature.return_annotation == 'Mesh':
        setattr(functions, name, napari_compatible(func))

        description = func.__doc__.split('\n')
        description.remove('') if '' in description else None
        description = [x.strip() for x in description][0]

        functions_to_add_to_yaml.append(
            {'command_name': 'napari-vedo-bridge.' + func.__name__,
                'python_name': 'napari_vedo_bridge.' + func.__name__,
                'description': description})

globals()['vedo_functions'] = functions

functions_to_add_to_yaml = pd.DataFrame(functions_to_add_to_yaml)
functions_to_add_to_yaml.head()

__module__ is not a callable
__doc__ is not a callable
edges is not a callable
cell_normals is not a callable


,command_name,python_name,description
0,napari-vedo-bridge.compute_normals,napari_vedo_bridge.compute_normals,Compute cell and vertex normals for the mesh.
1,napari-vedo-bridge.reverse,napari_vedo_bridge.reverse,Reverse the order of polygonal cells
2,napari-vedo-bridge.non_manifold_faces,napari_vedo_bridge.non_manifold_faces,Detect and (try to) remove non-manifold faces ...
3,napari-vedo-bridge.shrink,napari_vedo_bridge.shrink,Shrink the triangle polydata in the representa...
4,napari-vedo-bridge.cap,napari_vedo_bridge.cap,"Generate a ""cap"" on a clipped mesh, or caps sh..."


In [9]:
# Create a dictionary for name and display_name
metadata = {
    'name': 'napari-vedo-brdige',
    'display_name': 'Vedo'
}

# Create a dictionary for contributions
contributions = {
    'commands': [],
    'widgets': []
}

for index, row in functions_to_add_to_yaml.iterrows():
    command_entry = {
        'id': row['command_name'],
        'python_name': row['python_name'],
        'title': row['description']
    }

    widget_entry = {
        'command': row['command_name'],
        'autogenerate': True,
        'display_name': row['description']
    }

    contributions['commands'].append(command_entry)
    contributions['widgets'].append(widget_entry)

In [10]:
# Write the dictionary to a YAML file
with open('output.yaml', 'w') as yaml_file:
    yaml.dump(metadata, yaml_file, default_flow_style=False)
    yaml.dump(contributions, yaml_file, default_flow_style=False)